In [ ]:
# Zach Zager
# Yelp Data Wrangling and Organization
# March 2018

In [4]:
# pull in datasets
import pandas as pd
df = pd.read_csv("yelp-dataset/yelp_business.csv")
b = pd.read_csv("yelp-dataset/yelp_business_attributes.csv")
c = pd.read_csv("yelp-dataset/yelp_business_hours.csv")
d = pd.read_csv("yelp-dataset/yelp_checkin.csv")

In [5]:
# filter out non-US entries using US state abbreviations
state_list = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "DC", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]
df = df[df['state'].isin(state_list)]

In [6]:
# get rid of quotations marks from name and address fields
df['name'] = df['name'].str.replace('"','')
df['address'] = df['address'].str.replace('"','')

# merge “yelp_business_attributes" into main dataframe
df = df.merge(b, on='business_id', how='left')

# get rid of Na and NaN values
df = df.replace('Na','')
df = df.replace('NaN','')

In [7]:
# compile total days open, make column for total, merge days table into main

# convert day strings to binary values
c[['monday','tuesday','wednesday','thursday','friday','saturday','sunday']] = c[['monday','tuesday','wednesday','thursday','friday','saturday','sunday']] != 'None'

# add days open column, sum days of week binary values
c['days_open'] = c['monday'].astype('int64') + c['tuesday'].astype('int64') + c['wednesday'].astype('int64') + c['thursday'].astype('int64') + c['friday'].astype('int64') + c['saturday'].astype('int64') + c['sunday'].astype('int64')

# add “yelp_business_hours" into main dataframe
df = df.merge(c, on='business_id', how='left')

In [8]:
# define lists for parsing out
daysoftheweek = ['Mon','Tue','Wed','Thu','Fri','Sat','Sun']

timesoftheday = ['0:00','1:00','2:00','3:00','4:00','5:00','6:00','7:00','8:00',
                 '9:00','10:00','11:00','12:00','13:00','14:00','15:00','16:00',
                 '17:00','18:00','19:00','20:00','21:00','22:00','23:00']

In [10]:
# create new dataframe of each business id w/ hourly time columns

timecollist = [] # list to store columns
timecolheaderlist = [] # list to store headers (in order)
timecolheaderlist.append('checkins') # keep checkins column
for day in daysoftheweek:
    for time in timesoftheday:
        colname = day + time
        timecolheaderlist.append(colname)
        # get set of all corresponding day/hour values
        tempset = d[(d['weekday'] == day) & (d['hour'] == time)]
        # add checkin values to new column
        tempset.loc[:,colname] = tempset['checkins'].astype('int64')
        timecollist.append(tempset)

# combine all new columns
hourlyframe = pd.concat(timecollist)

In [11]:
# sum hourlyframe by business_id
hourlyframe = hourlyframe.groupby('business_id').sum() # sum columns
hourlyframe = hourlyframe[timecolheaderlist].reset_index() # reorganize columns

,business_id,checkins,Mon0:00,Mon1:00,Mon2:00,Mon3:00,Mon4:00,Mon5:00,Mon6:00,Mon7:00,...,Sun14:00,Sun15:00,Sun16:00,Sun17:00,Sun18:00,Sun19:00,Sun20:00,Sun21:00,Sun22:00,Sun23:00
0,--6MefnULPED_I942VcFNA,139,6.0,3.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,4.0,4.0,0.0,0.0,2.0,6.0,5.0
1,--7zmmkVg-IMGaXbuVd0SQ,153,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,5.0,4.0,4.0,2.0,6.0,2.0,0.0
2,--8LPVSo5i0Oo61X01sV9A,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,--9QQLMTbFzLJ_oT-ON3Xw,33,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,2.0,0.0,0.0,0.0
4,--9e1ONYQuAa-CB_Rrw7Tw,2568,9.0,39.0,59.0,46.0,48.0,26.0,10.0,1.0,...,1.0,1.0,3.0,1.0,9.0,19.0,24.0,9.0,10.0,3.0
5,--DaPTJW3-tB1vP-PfdTEg,89,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,3.0,3.0,1.0,2.0,0.0,0.0,0.0,0.0
6,--DdmeR16TRb3LsjG0ejrQ,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
7,--EX4rRznJrltyn-34Jz1w,12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
8,--FBCX-N37CMYDfs790Bnw,490,4.0,9.0,5.0,5.0,2.0,5.0,1.0,2.0,...,3.0,1.0,1.0,3.0,7.0,4.0,2.0,3.0,5.0,2.0
9,--FLdgM0GNpXVMn74ppCGw,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# merge hourlyframe into main dataframe
df = df.merge(hourlyframe, on='business_id', how='left')

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,...,Sun14:00,Sun15:00,Sun16:00,Sun17:00,Sun18:00,Sun19:00,Sun20:00,Sun21:00,Sun22:00,Sun23:00
0,FYWN1wneV18bWNgQjJ2GNg,Dental by Design,NaN,"4855 E Warner Rd, Ste B9",Ahwatukee,AZ,85044,33.330690,-111.978599,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,He-G7vWjzVUysIKrfNbPUQ,Stephen Szabo Salon,NaN,3101 Washington Rd,McMurray,PA,15317,40.291685,-80.104900,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,KQPW8lFf1y5BT2MxiSZ3QA,Western Motor Vehicle,NaN,"6025 N 27th Ave, Ste 1",Phoenix,AZ,85017,33.524903,-112.115310,1.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,8DShNS-LuFqpEWIp0HxijA,Sports Authority,NaN,"5000 Arizona Mills Cr, Ste 435",Tempe,AZ,85282,33.383147,-111.964725,3.0,...,0.0,1.0,0.0,0.0,2.0,7.0,0.0,3.0,1.0,4.0
4,PfOCPjBrlQAnz__NXj9h_w,Brick House Tavern + Tap,NaN,581 Howe Ave,Cuyahoga Falls,OH,44221,41.119535,-81.475690,3.5,...,0.0,2.0,4.0,6.0,6.0,5.0,6.0,5.0,8.0,1.0
5,kCoE3jvEtg6UVz5SOD3GVw,BDJ Realty,Summerlin,"2620 Regatta Dr, Ste 102",Las Vegas,NV,89128,36.207430,-115.268460,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,OD2hnuuTJI9uotcKycxg1A,Soccer Zone,NaN,"7240 W Lake Mead Blvd, Ste 4",Las Vegas,NV,89128,36.197484,-115.249660,1.5,...,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0
7,EsMcGiZaQuG1OOvL9iUFug,Any Given Sundae,NaN,2612 Brandt School Rd,Wexford,PA,15090,40.615102,-80.091349,5.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0
8,TGWhGNusxyMaA4kQVBNeew,Detailing Gone Mobile,NaN,,Henderson,NV,89014,36.055825,-115.046350,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,XOSRcvtaKc_Q5H1SAzN20A,East Coast Coffee,NaN,737 West Pike St,Houston,PA,15342,40.241548,-80.212815,4.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df.to_csv('yelp_data.csv')